In [1]:
import random
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import preprocessing
from scipy import sparse
from tqdm import tqdm
from collections import Counter

In [2]:
train_pd = pd.read_csv("./data/train.csv", encoding="utf-8")

print("get word_list & vocab_list")
word_list = []
vocab_list = []
for line in tqdm(train_pd.value):
    word_list.append([w for w in line])
    vocab_list.extend([w for w in line])
    
print("get vocab")
c = Counter(vocab_list)
vocab = np.array(list(c.keys()))
vocab.sort()
print(vocab)

print("lable encoder vocab")
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(vocab)
print(label_encoder.classes_)
# label_encoder.transform(vocab)

print("label encoder word_list")
word_labelencoder = [list(label_encoder.transform(w)) for w in word_list]
word_size = [len(i) for i in word_labelencoder]

print("balanced pos & neg data")
train_size_pd = pd.DataFrame(word_size, columns=['len'])
train_size_pd['label'] = train_pd.label
# train_size_pd.describe()
train_size_pd = train_size_pd[train_size_pd.len < 500]
print(Counter(train_size_pd.label))

pos_index = train_size_pd[train_size_pd.label == 1].index.values
random.shuffle(pos_index)
print(pos_index, pos_index.shape)

neg_index = train_size_pd[train_size_pd.label == 0].index.values
neg_index = np.random.choice(neg_index, size=4000-1973)
print(neg_index, neg_index.shape)

balance_train_index = np.append(pos_index,neg_index)
random.shuffle(balance_train_index)
print(balance_train_index, balance_train_index.shape)

print("get data")
x_batch = np.array(word_labelencoder.copy())
x_batch = x_batch[balance_train_index]
y_batch = np.array(train_pd.label.copy())
y_batch = y_batch[balance_train_index]
x_batch_size = [len(i) for i in x_batch]
print(x_batch.shape, y_batch.shape, len(x_batch_size))

print("padding & one-hot x data")
max_size = np.max(x_batch_size)
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
#     list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)
x_batch_pad = [np.eye(len(vocab))[item] for item in x_batch_pad]
   
print("padding & one-hot y data")
y_batch_pad = [np.eye(2)[item] for item in y_batch]
y_batch_pad = [list(i) for i in y_batch_pad]

print("success")

get word_list & vocab_list


100%|████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 101760.35it/s]


get vocab
['%' '&' '*' '+' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '=' 'A'
 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S'
 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j'
 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
lable encoder vocab
['%' '&' '*' '+' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '=' 'A'
 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S'
 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j'
 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
label encoder word_list
balanced pos & neg data
Counter({0: 27159, 1: 1973})
[16990 13224 19547 ..., 21312 24136 25566] (1973,)
[11951  9750 29412 ...,  4893  9309 20971] (2027,)
[16191  3846 29602 ..., 13505 19548 14360] (4000,)
get data
(4000,) (4000,) 4000
padding & one-hot x data


100%|███████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 37646.96it/s]


padding & one-hot y data
success


In [3]:
class Batcher():
    def __init__(self, x, y, x_batch_size):
        self.split_size = int(len(x)*0.8)
        self.train_x = x[:self.split_size]
        self.train_y = y[:self.split_size]
        self.train_size = x_batch_size[:self.split_size]
        self.test_x = x[self.split_size:]
        self.test_y = y[self.split_size:]
        self.test_size = x_batch_size[self.split_size:]
        self.start = 0
    def next_batch(self, batch_size):
        s_index = self.start
        e_index = self.start + batch_size
        if e_index >= self.split_size:
            self.start = 0
            s_index = self.start
            e_index = self.start + batch_size
        self.start = e_index
        return self.train_x[s_index:e_index], self.train_y[s_index:e_index], self.train_size[s_index:e_index]

In [4]:
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

In [5]:
batch_data, batch_labels, batch_seqlen = sqli_batch.next_batch(10)

In [6]:
batch_data[:2]

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]])]

In [7]:
batch_labels

[[0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0]]

In [15]:
batch_seqlen

[374, 110, 104, 404, 325, 56, 63, 263, 72, 46]

In [24]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_steps = 500
batch_size = 100
display_step = 50

# Network Parameters
seq_max_len = max_size # Sequence max length
n_hidden = 128 # hidden layer num of features
n_classes = 2 # linear sequence or not

# trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
# testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, len(vocab)])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None,])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
#     x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    encoder_fw = tf.contrib.rnn.GRUCell(n_hidden)
    encoder_bw = tf.contrib.rnn.GRUCell(n_hidden)
#     encoder_fw = tf.contrib.rnn.DropoutWrapper(encoder_fw, output_keep_prob=self.output_keep_prob)
#     encoder_bw = tf.contrib.rnn.DropoutWrapper(encoder_bw, output_keep_prob=self.output_keep_prob)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
#     outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
#                                 sequence_length=seqlen)
    with tf.variable_scope("bi-GRU") as scope:
        ((encoder_fw_output, encoder_bw_output), 
             (encoder_fw_state, encoder_bw_state)) = (
                tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_fw, 
                                                cell_bw=encoder_bw, 
                                                inputs=x,
                                                sequence_length=seqlen,
                                                dtype=tf.float32)
            )

        outputs = tf.concat((encoder_fw_output, encoder_bw_output), 2) 
        #[batch_size, max_time, 2 * encoder_hidden_size]
        states = tf.concat((encoder_fw_state, encoder_bw_state), 1)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
#     outputs = tf.stack(outputs)
#     outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, 2*n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [26]:
LOG_DIR = "./biGRU/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

# Start training
with tf.Session() as sess:
    tf.summary.scalar("loss", cost)
    tf.summary.scalar("acc", accuracy)
        
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(LOG_DIR + "train/", sess.graph)

    # Run the initializer
#     sess.run(init)
    saver.restore(sess, module_file)

    for step in range(1, training_steps + 1):
        # Get batch data
        batch_x, batch_y, batch_seqlen = sqli_batch.next_batch(batch_size)
        # Run optimization op (backprop)
        summary_ = sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, seqlen: batch_seqlen})
#         train_writer.add_summary(summary_, step)
        
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            
            # Calculate accuracy
            test_data = sqli_batch.test_x
            test_label = sqli_batch.test_y
            test_seqlen = sqli_batch.test_size
            test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
            print("Testing Accuracy:", test_acc)
        if step % 5 == 0:
            saver.save(sess, LOG_DIR+"sqli.ckpt", global_step=step)

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./biGRU/sqli.ckpt-315
Step 100, Minibatch Loss= 0.211029, Training Accuracy= 0.94000
Testing Accuracy: 0.96
Step 5000, Minibatch Loss= 0.194533, Training Accuracy= 0.93000
Testing Accuracy: 0.95625
Step 10000, Minibatch Loss= 0.170305, Training Accuracy= 0.96000
Testing Accuracy: 0.9625
Step 15000, Minibatch Loss= 0.054785, Training Accuracy= 0.99000
Testing Accuracy: 0.96
Step 20000, Minibatch Loss= 0.115224, Training Accuracy= 0.95000
Testing Accuracy: 0.9625
Step 25000, Minibatch Loss= 0.146522, Training Accuracy= 0.95000
Testing Accuracy: 0.965
Step 30000, Minibatch Loss= 0.165512, Training Accuracy= 0.91000
Testing Accuracy: 0.95
Step 35000, Minibatch Loss= 0.050726, Training Accuracy= 0.99000
Testing Accuracy: 0.96375
Step 40000, Minibatch Loss= 0.082588, Training Accuracy= 0.98000
Testing Accuracy: 0.96875
Step 45000, Minibatch Loss= 0.093765, Training Accuracy= 0.97000
Testing Accuracy: 0.96625
Step 50000, Minibatch Loss= 0.096553, Trai

In [27]:
LOG_DIR = "./biGRU/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

test_pred = ''
# Start training
with tf.Session() as sess:
    
    saver.restore(sess, module_file)

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_pred = sess.run(pred, feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./biGRU/sqli.ckpt-500
Testing Accuracy: 0.95625


In [28]:
test_pred_label = np.argmax(test_pred, 1)
test_real_label = np.argmax(test_label, 1)
test_diff = test_pred_label != test_real_label
test_diff_index = np.where(test_diff == True)[0]
print("diff len:", len(test_diff_index))

x_batch_lbl = np.array(word_labelencoder.copy())
x_batch_lbl = x_batch_lbl[balance_train_index]
word_list_array = np.array(word_list)
test_data = word_list_array[balance_train_index][int(0.8*len(balance_train_index)):]
print("get test data")

diff_data = test_data[test_diff_index]
diff_data_str = []
for data in diff_data:
    diff_data_str.append("".join(data))
#     print("".join(data))
print("get diff raw data")

test_data_all = np.array(train_pd.label.copy())
test_data_y = test_data_all[balance_train_index][int(0.8*len(balance_train_index)):]

print("test_pred_label", test_pred_label[test_diff_index])
print("test_real_label", test_real_label[test_diff_index])
print("    test_data_y", test_data_y[test_diff_index])

diff len: 35
get test data
get diff raw data
test_pred_label [0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
test_real_label [1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
    test_data_y [1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]


In [29]:
diff_pd = pd.DataFrame(diff_data_str, columns=['urldata'])
diff_pd['pred'] = test_pred_label[test_diff_index]
diff_pd['real'] = test_real_label[test_diff_index]
diff_pd['label'] = test_data_y[test_diff_index]
diff_pd.to_csv("./data/004#biGRU_diff.csv", index=False)

In [30]:
diff_pd

,urldata,pred,real,label
0,keyone=7976314481&keytwo=foo%2520AND%25201427%...,0,1,1
1,keyone=DF102861-F280-48A2-AAB8-8FD49E464E90&ke...,1,0,0
2,keyone=16-999.9+union+select+1%2Cconcat_ws%28u...,0,1,1
3,keyone=1478158744072_249&keytwo=1478073231783_...,0,1,1
4,keyone=-1+UNION+ALL+SELECT+1%2C2%2C3&keytwo=14...,0,1,1
5,keyone=-2551%2527%2520WHERE%25205598%253D5598%...,0,1,1
6,keyone=-1+union+select+1%2C2%2C3%2C4%2CUSER%28...,0,1,1
7,keyone=1478670190391_179&keytwo=1001-%5CN+unio...,0,1,1
8,keyone=dc11bc8986524b6adbce87e84a81c2558523238...,1,0,0
9,keyone=7971664867&keytwo=VdB25Ot2q&keythree=A+...,0,1,1
